In [1]:
import os 
os.chdir("../")

In [4]:
from huggingface_hub import hf_hub_download
from ultralytics import YOLO
from supervision import Detections
from PIL import Image, ImageDraw

# download and load model
model = YOLO("models/yolo_model/model.pt")


In [15]:
image_path = "training/pic1.png"
image = Image.open(image_path)

# run inference
output = model(image)
detections = Detections.from_ultralytics(output[0])

print( f"bound box = {detections.xyxy}")

# draw bounding boxes
draw = ImageDraw.Draw(image)
for box in detections.xyxy:  # corrected to detections.xyxy
    # box coordinates
    x1, y1, x2, y2 = box
    draw.rectangle([x1, y1, x2, y2], outline="red", width=3)


0: 608x640 29 FACEs, 257.4ms
Speed: 9.0ms preprocess, 257.4ms inference, 5.5ms postprocess per image at shape (1, 3, 608, 640)
bound box = [[     259.41      308.27      291.97      342.43]
 [     790.16      307.21      825.19      348.24]
 [      453.5      328.56      481.96      364.92]
 [     399.95      314.73      431.59      352.02]
 [     61.833      347.77      93.337      384.23]
 [     680.46       343.1       713.2      377.78]
 [     497.69      335.04      527.55      370.52]
 [     565.13      461.96       595.7      496.66]
 [     124.37      327.43       155.3      363.51]
 [     304.06      324.31      332.47      356.55]
 [     20.009      332.62       47.02      371.12]
 [     647.18      335.49      675.49      372.27]
 [     737.05      346.62      768.44      381.41]
 [     613.18      305.45      640.09      340.72]
 [     561.44      328.86      590.18      364.48]
 [      342.1      330.22      370.16      364.99]
 [     469.81      479.59      503.15      5

In [ ]:
# https://yolov8.org/how-to-use-fine-tune-yolov8/

In [ ]:
from ultralytics import YOLO

# Load the pre-trained YOLOv8 model
model = YOLO("yolov8n.pt")  # Use yolov8n.pt, yolov8s.pt, etc. depending on the size

# Train the model on your dataset
model.train(
    data="data.yaml",  # Path to your data.yaml file
    epochs=50,         # Number of training epochs
    imgsz=640,         # Image size
    batch=16,          # Batch size
    name="yolov8_face_finetune",  # Experiment name
    pretrained=True    # Start from the pre-trained weight
)

In [4]:
import cv2

# Load the pre-trained Haar Cascade model for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Read the image
# Use double backslashes or a raw string to specify the path correctly
img = cv2.imread(r'research\pic1.png')  # or use 'research\\pic1.png'

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Detect faces in the image
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

# Draw rectangles around each face
for (x, y, w, h) in faces:
    cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)

# Display the result
cv2.imshow('Detected Faces', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [1]:
import cv2
import csv
import os

# Parameters
num_classes = 2
num_images = 50
output_dir = "captured_images"
os.makedirs(output_dir, exist_ok=True)

# CSV file for bounding box annotations
annotations_file = os.path.join(output_dir, "annotations.csv")

# Load the pre-trained Haar Cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Open the CSV file for writing
with open(annotations_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write the header for CSV
    writer.writerow(["image", "class", "xmin", "ymin", "xmax", "ymax"])

    # Loop through each class
    for class_idx in range(num_classes):
        class_name = input(f"Enter name for class {class_idx + 1}: ")
        print(f"Press 's' to start capturing {num_images} images for class '{class_name}'...")

        captured_count = 0  # Counter for images in the current class
        start_capturing = False

        while captured_count < num_images:
            # Capture frame-by-frame
            ret, frame = cap.read()
            if not ret:
                break

            # Convert frame to grayscale (required for face detection)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Detect faces in the frame
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

            # Draw bounding boxes around detected faces
            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

                # If capturing is started, save the image and record the bounding box
                if start_capturing and captured_count < num_images:
                    # Define bounding box coordinates
                    xmin, ymin, xmax, ymax = x, y, x + w, y + h
                    img_filename = f"{output_dir}/frame_class_{class_name}_{captured_count}.jpg"

                    # Save the image with bounding box
                    cv2.imwrite(img_filename, frame)
                    
                    # Save annotation to CSV
                    writer.writerow([img_filename, class_name, xmin, ymin, xmax, ymax])

                    captured_count += 1
                    print(f"Captured image {captured_count} for class '{class_name}'")

            # Display the frame with bounding boxes
            cv2.imshow('Face Detection', frame)

            # Start capturing when "s" is pressed
            key = cv2.waitKey(1) & 0xFF
            if key == ord('s'):
                start_capturing = True
                print(f"Started capturing for class '{class_name}'")
            elif key == ord('q'):
                # Exit the program if "q" is pressed
                break

        # Reset start_capturing for the next class
        start_capturing = False
        print(f"Finished capturing for class '{class_name}'")

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


Press 's' to start capturing 50 images for class 'rahul'...
Started capturing for class 'rahul'
Captured image 1 for class 'rahul'
Captured image 2 for class 'rahul'
Captured image 3 for class 'rahul'
Captured image 4 for class 'rahul'
Captured image 5 for class 'rahul'
Captured image 6 for class 'rahul'
Captured image 7 for class 'rahul'
Captured image 8 for class 'rahul'
Captured image 9 for class 'rahul'
Captured image 10 for class 'rahul'
Captured image 11 for class 'rahul'
Captured image 12 for class 'rahul'
Captured image 13 for class 'rahul'
Captured image 14 for class 'rahul'
Captured image 15 for class 'rahul'
Captured image 16 for class 'rahul'
Captured image 17 for class 'rahul'
Captured image 18 for class 'rahul'
Captured image 19 for class 'rahul'
Captured image 20 for class 'rahul'
Captured image 21 for class 'rahul'
Captured image 22 for class 'rahul'
Captured image 23 for class 'rahul'
Captured image 24 for class 'rahul'
Captured image 25 for class 'rahul'
Captured imag